In [62]:
import collections
import math
import random
import jieba
import numpy as np
from six.moves import xrange
import tensorflow as tf

In [63]:
def read_data():
    """
    对要训练的文本进行处理，最后把文本的内容的所有词放在一个列表中
    """
    #读取停用词
    stop_words = []
    with open('C:/Users/dell/Desktop/word2vec-tensorflow-master/stop_words.txt',"r",encoding="UTF-8") as f:
        line = f.readline()
        while line:
            stop_words.append(line[:-1])
            line = f.readline()
    stop_words = set(stop_words)
    print('停用词读取完毕，共{n}个词'.format(n=len(stop_words)))

    # 读取文本，预处理，分词，得到词典
    raw_word_list = []
    with open('C:/Users/dell/Desktop/word2vec-tensorflow-master/doupocangqiong.txt',"r", encoding='UTF-8') as f:
        line = f.readline()
        while line:
            while '\n' in line:
                line = line.replace('\n','')
            while ' ' in line:
                line = line.replace(' ','')
            while '…' in line:
                line = line.replace('…','')
            while '”' in line:
                line = line.replace('”','')
            if len(line)>0: # 如果句子非空
                raw_words = list(jieba.cut(line,cut_all=False))
                raw_word_list.extend(raw_words)
            line=f.readline()
    return raw_word_list

In [64]:
word_list =read_data()


停用词读取完毕，共1893个词


In [65]:
vocabulary_size = 50000

def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    print("count",len(count))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [66]:
data, count, dictionary, reverse_dictionary = build_dataset(word_list)

count 50000


In [67]:
data_index = 0

In [68]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],'->', labels[i, 0], reverse_dictionary[labels[i, 0]])

32417 《 -> 12889 斗破
32417 《 -> 32416 ﻿
12889 斗破 -> 32417 《
12889 斗破 -> 7446 苍穹
7446 苍穹 -> 12889 斗破
7446 苍穹 -> 32418 》
32418 》 -> 7446 苍穹
32418 》 -> 25329 第一章


In [69]:
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],'->', labels[i, 0], reverse_dictionary[labels[i, 0]])

32417 《 -> 12889 斗破
32417 《 -> 32416 ﻿
12889 斗破 -> 32417 《
12889 斗破 -> 7446 苍穹
7446 苍穹 -> 12889 斗破
7446 苍穹 -> 32418 》
32418 》 -> 7446 苍穹
32418 》 -> 25329 第一章


In [70]:
batch_size = 128
embedding_size = 128  
skip_window = 1       
num_skips = 2         
valid_size = 9       
valid_window = 100  
num_sampled = 64   

In [71]:
valid_word = ['萧炎','灵魂','火焰','萧薰儿','药老','天阶',"云岚宗","乌坦城","惊诧"]
valid_examples =[dictionary[li] for li in valid_word]

In [72]:
valid_examples

[8, 102, 75, 7641, 13733, 2015, 304, 1669, 1987]

In [73]:
graph = tf.Graph()

In [74]:
with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    with tf.device('/cpu:0'):

        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)


        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]),dtype=tf.float32)

    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                         biases=nce_biases, 
                                         inputs=embed, 
                                         labels=train_labels,
                                         num_sampled=num_sampled, 
                                         num_classes=vocabulary_size))


    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    #维度 [embeddings_size,1] 
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    #维度 [valid_size,embeddings_size] 
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    #维度 [valid_size,vocabulary_size]
    init = tf.global_variables_initializer()
    print(normalized_embeddings)

Tensor("truediv:0", shape=(50000, 128), dtype=float32)


In [75]:
num_steps= 3000000

In [134]:
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")
    saver=tf.train.Saver(max_to_keep=10)
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}


        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000

            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[:top_k]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
            
                print(log_str)
            saver.save(session,'D:/sess/',global_step=1000)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  289.82305908203125
Nearest to 萧炎: 萧炎, 先打, 只留, 听错, 冰雾, 一只, 这云, 血浪,
Nearest to 灵魂: 灵魂, 陆牧, 探亲, 怪力, 费去, 无害, 回龙, 心急火燎,
Nearest to 火焰: 火焰, 一前一后, 漫场, 章大, 以三人, 流窜, 内脏, 这雷,
Nearest to 萧薰儿: 萧薰儿, 上绘, 师欠, 请动, 想到, 看着办, 门门主, 攻城,
Nearest to 药老: 药老, 若干, 天而起, 咆哮声, 小规模, 这不怪, 电击, 砸,
Nearest to 天阶: 天阶, 爬上来, 周折, 紧绷着, 留印, 可蛇, 沉默, 怨言,
Nearest to 云岚宗: 云岚宗, 老落, 巨熊, 那人醒, 老杂, 买主, 坑痕, 三心二意,
Nearest to 乌坦城: 乌坦城, 差, 十六七, 第六, 唉, 略有, 成蓝, 接战,
Nearest to 惊诧: 惊诧, 本皇, 云韵玉, 眼珠子, 多数, 不早, 回过, 玉袖,
Average loss at step  2000 :  110.82173763370514
Average loss at step  4000 :  50.454421230316164
Average loss at step  6000 :  31.472390153884888
Average loss at step  8000 :  22.373685292959212
Average loss at step  10000 :  17.700019909262657
Nearest to 萧炎: 萧炎, 她, 他, 淫, 俏皮, 我, 因此, 率先,
Nearest to 灵魂: 灵魂, 无害, 陆牧, 皱着眉头, 稳固, 倩影, 划下, ，,
Nearest to 火焰: 火焰, 一前一后, 想着, 章大, 波纹, 庆贺, 咬牙, 无用,
Nearest to 萧薰儿: 萧薰儿, 想到, 支持, 请动, 高, 脑中, 瓶, 微竖,
Nearest to 药老: 药老, 咆哮声, 砸, 挥, 速战速决, 雅涵, 看看, 双方,
Nearest to 天阶: 天

Average loss at step  120000 :  3.849496534705162
Nearest to 萧炎: 萧炎, 他, 她, 紫研, 后者, 他们, 古河, 柳擎,
Nearest to 灵魂: 灵魂, 能量, 陆牧, 十二岁, 倩影, 划下, 千层, 筛选,
Nearest to 火焰: 火焰, 能量, 波纹, 黑雾, 光芒, 一前一后, 斗气, 想着,
Nearest to 萧薰儿: 萧薰儿, 请动, 支持, 愿助, 想到, 脑中, 上绘, 微竖,
Nearest to 药老: 药老, 看看, 咆哮声, 若干, 速战速决, 这不怪, 解决, 天而起,
Nearest to 天阶: 天阶, 紧绷着, 玉琢, 无法形容, 三滴, 沉默, 周折, 添进,
Nearest to 云岚宗: 云岚宗, 萧家, 巨熊, 三心二意, 低阶, 炼, 大不一样, 四个,
Nearest to 乌坦城: 乌坦城, 差, 十六七, 唉, 未加, 略有, 坑坑洼洼, 第六,
Nearest to 惊诧: 惊诧, 本皇, 眼珠子, 荑, 阴沉, 异火太过, 其望, 错愕,
Average loss at step  122000 :  3.850006876349449
Average loss at step  124000 :  3.788005366206169
Average loss at step  126000 :  3.8207023181915285
Average loss at step  128000 :  3.842789968729019
Average loss at step  130000 :  3.738982696056366
Nearest to 萧炎: 萧炎, 他, 她, 紫研, 后者, 他们, 小医仙, 前者,
Nearest to 灵魂: 灵魂, 能量, 十二岁, 陆牧, 千层, 倩影, 划下, 筛选,
Nearest to 火焰: 火焰, 能量, 黑雾, 波纹, 光芒, 一前一后, 斗气, 不必要,
Nearest to 萧薰儿: 萧薰儿, 请动, 支持, 愿助, 想到, 上绘, 脑中, 微竖,
Nearest to 药老: 药老, 看看, 咆哮声, 若干, 这不怪, 速战速决, 解决, 天而起,
Nearest to

Average loss at step  234000 :  3.639735607147217
Average loss at step  236000 :  3.5731453380584717
Average loss at step  238000 :  3.579556508421898
Average loss at step  240000 :  3.58646751499176
Nearest to 萧炎: 萧炎, 他, 后者, 紫研, 她, 小医仙, 他们, 柳擎,
Nearest to 灵魂: 灵魂, 能量, 十二岁, 陆牧, 斗气, 千层, 无力, 大地,
Nearest to 火焰: 火焰, 能量, 黑雾, 波纹, 光芒, 斗气, 异火, 粉,
Nearest to 萧薰儿: 萧薰儿, 请动, 支持, 愿助, 脑中, 立, 微竖, 要是,
Nearest to 药老: 药老, 看看, 若干, 这不怪, 咆哮声, 速战速决, 天而起, 小规模,
Nearest to 天阶: 天阶, 无法形容, 斗王, 玉琢, 三滴, 高阶, 躲闪, 周折,
Nearest to 云岚宗: 云岚宗, 萧家, 云山, 巨熊, 三心二意, 深藏, 老落, 这锁魂,
Nearest to 乌坦城: 乌坦城, 十六七, 差, 唉, 密室, 古族, 未加, 淤血,
Nearest to 惊诧: 惊诧, 阴沉, 不难想象, 本皇, 错愕, 异火太过, 聪明, 其望,
Average loss at step  242000 :  3.5947490878105164
Average loss at step  244000 :  3.5722304282188415
Average loss at step  246000 :  3.563333023428917
Average loss at step  248000 :  3.585473069906235
Average loss at step  250000 :  3.7608917324543
Nearest to 萧炎: 萧炎, 他, 后者, 紫研, 她, 小医仙, 他们, 前者,
Nearest to 灵魂: 灵魂, 能量, 十二岁, 陆牧, 无力, 千层, 蛮不讲理, 无形,
Nearest to 火焰

Average loss at step  352000 :  3.6331444530487063
Average loss at step  354000 :  3.6144817551374437
Average loss at step  356000 :  3.559466662526131
Average loss at step  358000 :  3.5615607783794405
Average loss at step  360000 :  3.5482164018154143
Nearest to 萧炎: 萧炎, 他, 后者, 她, 紫研, 前者, 柳擎, 林焱,
Nearest to 灵魂: 灵魂, 能量, 十二岁, 陆牧, 无力, 无形, 回音, 不在话下,
Nearest to 火焰: 火焰, 能量, 黑雾, 异火, 波纹, 粉, 光芒, 火浪,
Nearest to 萧薰儿: 萧薰儿, 请动, 支持, 脑中, 要是, 微竖, 师欠, 立,
Nearest to 药老: 药老, 若干, 看看, 这不怪, 速战速决, 小规模, 咆哮声, 天而起,
Nearest to 天阶: 天阶, 玄阶, 高阶, 地阶, 斗王, 无法形容, 功法, 人类,
Nearest to 云岚宗: 云岚宗, 萧家, 云山, 丹塔, 深藏, 魂族, 墨家, 院,
Nearest to 乌坦城: 乌坦城, 十六七, 古族, 差, 密室, 地实火, 淤血, 唉,
Nearest to 惊诧: 惊诧, 不难想象, 错愕, 异火太过, 本皇, 阴沉, 冷意, 其望,
Average loss at step  362000 :  3.588267262339592
Average loss at step  364000 :  3.6097903348207474
Average loss at step  366000 :  3.5912950961589813
Average loss at step  368000 :  3.5811302597522734
Average loss at step  370000 :  3.5796384452581407
Nearest to 萧炎: 萧炎, 后者, 他, 紫研, 她, 韩枫, 林焱, 柳擎,
Nearest 

Average loss at step  472000 :  3.5576626801490785
Average loss at step  474000 :  3.5210666913986204
Average loss at step  476000 :  3.497548829317093
Average loss at step  478000 :  3.483746608257294
Average loss at step  480000 :  3.531427146792412
Nearest to 萧炎: 萧炎, 他, 后者, 韩枫, 她, 萧厉, 紫研, 柳擎,
Nearest to 灵魂: 灵魂, 能量, 十二岁, 无力, 陆牧, 不在话下, 斗气, 无形,
Nearest to 火焰: 火焰, 能量, 黑雾, 异火, 波纹, 火浪, 光点, 粉,
Nearest to 萧薰儿: 萧薰儿, 请动, 支持, 微竖, 师欠, 要是, 脑中, 娃娃亲,
Nearest to 药老: 药老, 这不怪, 若干, 看看, 速战速决, 小规模, 快进去, 天而起,
Nearest to 天阶: 天阶, 玄阶, 地阶, 高阶, 斗王, 功法, 无法形容, 人类,
Nearest to 云岚宗: 云岚宗, 萧家, 云山, 丹塔, 魔炎谷, 深藏, 院, 魂族,
Nearest to 乌坦城: 乌坦城, 十六七, 古族, 天焚, 密室, 地实火, 石台, 遗迹,
Nearest to 惊诧: 惊诧, 错愕, 不难想象, 阴沉, 冷意, 异火太过, 本皇, 其望,
Average loss at step  482000 :  3.503668216109276
Average loss at step  484000 :  3.432021227955818
Average loss at step  486000 :  3.490680305957794
Average loss at step  488000 :  3.4463001148700716
Average loss at step  490000 :  3.4556561740636824
Nearest to 萧炎: 萧炎, 他, 后者, 紫研, 萧厉, 柳擎, 韩枫, 前者,
Neares

...

Average loss at step  2512000 :  3.237524136543274
Average loss at step  2514000 :  3.235455717086792
Average loss at step  2516000 :  3.216260867655277
Average loss at step  2518000 :  3.218270375967026
Average loss at step  2520000 :  3.2842033326625826
Nearest to 萧炎: 萧炎, 他, 古元, 萧厉, 紫研, 蝎山, 烛坤, 魂玉,
Nearest to 灵魂: 灵魂, 能量, 斗气, 天凰, 尽量少, 赏赐, 毁灭, 力量,
Nearest to 火焰: 火焰, 黑雾, 异火, 能量, 火苗, 冰层, 火海, 浓雾,
Nearest to 萧薰儿: 萧薰儿, 凌影, 雪魅, 微竖, 魂厉, 儿沉, 多说无益, 老狮,
Nearest to 药老: 药老, 快进去, 这不怪, 老点, 一搏, 五百八十五, 两败俱伤, 若干,
Nearest to 天阶: 天阶, 玄阶, 地阶, 七品, 高阶, 炼, 风属, 隐世,
Nearest to 云岚宗: 云岚宗, 磐门, 萧家, 丹塔, 曹家, 萧族, 墨家, 叶家,
Nearest to 乌坦城: 乌坦城, 加玛, 叶城, 石漠, 柳家, 帝国, 天焚, 崂,
Nearest to 惊诧: 惊诧, 诧异, 错愕, 愕然, 惊异, 惊讶, 讶异, 惊疑,
Average loss at step  2522000 :  3.218758915781975
Average loss at step  2524000 :  3.2499427573680877
Average loss at step  2526000 :  3.2241209481954574
Average loss at step  2528000 :  3.226297771811485
Average loss at step  2530000 :  3.2181236629486083
Nearest to 萧炎: 萧炎, 他, 后者, 前者, 古元, 蝎山, 木战, 穆力,
Near

Average loss at step  2632000 :  3.216937479734421
Average loss at step  2634000 :  3.252748257756233
Average loss at step  2636000 :  3.2618943828344347
Average loss at step  2638000 :  3.2271838596463205
Average loss at step  2640000 :  3.238586430847645
Nearest to 萧炎: 萧炎, 他, 木战, 蝎山, 后者, 韩枫, 萧厉, 紫研,
Nearest to 灵魂: 灵魂, 斗气, 能量, 赏赐, 回采, 力量, 龙凰, 天凰,
Nearest to 火焰: 火焰, 能量, 火苗, 异火, 冰层, 黑雾, 波纹, 火海,
Nearest to 萧薰儿: 萧薰儿, 凌影, 雪魅, 魂厉, 微竖, 儿沉, 老狮, 多说无益,
Nearest to 药老: 药老, 快进去, 这不怪, 一搏, 老点, 五百八十五, 两败俱伤, 对战,
Nearest to 天阶: 天阶, 玄阶, 地阶, 七品, 高阶, 隐世, 风属, 那紫火,
Nearest to 云岚宗: 云岚宗, 磐门, 丹塔, 萧家, 曹家, 慕兰谷, 宗内, 炎盟,
Nearest to 乌坦城: 乌坦城, 加玛, 叶城, 石漠, 柳家, 天焚, 崂, 帝国,
Nearest to 惊诧: 惊诧, 诧异, 错愕, 愕然, 惊异, 惊讶, 惊疑, 讶异,
Average loss at step  2642000 :  3.288380017280579
Average loss at step  2644000 :  3.2348693066835406
Average loss at step  2646000 :  3.219360319495201
Average loss at step  2648000 :  3.2161436045765877
Average loss at step  2650000 :  3.241530198574066
Nearest to 萧炎: 萧炎, 萧厉, 古元, 韩枫, 他, 柳擎, 萧鼎, 后者,
Ne

Average loss at step  2752000 :  3.242518990278244
Average loss at step  2754000 :  3.215767756342888
Average loss at step  2756000 :  3.183086175084114
Average loss at step  2758000 :  3.2219826033115386
Average loss at step  2760000 :  3.1882505774497987
Nearest to 萧炎: 萧炎, 萧厉, 古元, 他, 蝎山, 林焱, 柳擎, 萧鼎,
Nearest to 灵魂: 灵魂, 斗气, 能量, 赏赐, 回采, 力量, 毁灭, 尽量少,
Nearest to 火焰: 火焰, 能量, 异火, 黑雾, 火海, 冰层, 浓雾, 火苗,
Nearest to 萧薰儿: 萧薰儿, 凌影, 雪魅, 魂厉, 微竖, 自信, 儿沉, 老狮,
Nearest to 药老: 药老, 快进去, 这不怪, 一搏, 老点, 五百八十五, 对战, 两败俱伤,
Nearest to 天阶: 天阶, 玄阶, 地阶, 七品, 高阶, 隐世, 那紫火, 风属,
Nearest to 云岚宗: 云岚宗, 磐门, 萧家, 丹塔, 曹家, 炎盟, 宗内, 萧族,
Nearest to 乌坦城: 乌坦城, 加玛, 叶城, 石漠, 柳家, 崂, 帝国, 天焚,
Nearest to 惊诧: 惊诧, 诧异, 错愕, 愕然, 惊异, 惊讶, 惊疑, 讶异,
Average loss at step  2762000 :  3.2234631844758987
Average loss at step  2764000 :  3.2392252415418623
Average loss at step  2766000 :  3.2317940610647202
Average loss at step  2768000 :  3.1948226910829542
Average loss at step  2770000 :  3.2191985223293305
Nearest to 萧炎: 萧炎, 他, 古元, 萧厉, 柳擎, 魂玉, 韩枫, 蝎山,
N

Average loss at step  2872000 :  3.230386453807354
Average loss at step  2874000 :  3.2545292296409607
Average loss at step  2876000 :  3.2091319835186005
Average loss at step  2878000 :  3.2155885705947878
Average loss at step  2880000 :  3.200186335325241
Nearest to 萧炎: 萧炎, 他, 林焱, 穆力, 紫研, 古元, 蝎山, 萧厉,
Nearest to 灵魂: 灵魂, 斗气, 能量, 回采, 赏赐, 尽量少, 力量, 无形,
Nearest to 火焰: 火焰, 黑雾, 冰层, 能量, 火苗, 异火, 浓雾, 火海,
Nearest to 萧薰儿: 萧薰儿, 凌影, 雪魅, 魂厉, 微竖, 她, 儿沉, 木,
Nearest to 药老: 药老, 快进去, 一搏, 这不怪, 老点, 五百八十五, 二章, 对战,
Nearest to 天阶: 天阶, 玄阶, 地阶, 那紫火, 七品, 风属, 隐世, 高阶,
Nearest to 云岚宗: 云岚宗, 磐门, 萧家, 丹塔, 曹家, 宗内, 院, 炎盟,
Nearest to 乌坦城: 乌坦城, 加玛, 叶城, 石漠, 崂, 柳家, 帝国, 塔,
Nearest to 惊诧: 惊诧, 诧异, 错愕, 愕然, 惊异, 惊讶, 惊疑, 讶异,
Average loss at step  2882000 :  3.2088233531713484
Average loss at step  2884000 :  3.196871001362801
Average loss at step  2886000 :  3.2410333248376846
Average loss at step  2888000 :  3.246656409859657
Average loss at step  2890000 :  3.2193973361849784
Nearest to 萧炎: 萧炎, 他, 木战, 萧厉, 范痨, 韩枫, 穆力, 柳擎,
Nearest 

Average loss at step  2992000 :  3.2262270785570144
Average loss at step  2994000 :  3.2310980125665663
Average loss at step  2996000 :  3.2614072699546814
Average loss at step  2998000 :  3.2216616208553313


读取模型

In [76]:
saver= tf.train.import_meta_graph('D:/sess/-1000.meta')

In [84]:
with tf.Session() as sess:

    # load the meta graph and weights
    saver = tf.train.import_meta_graph('D:/sess/-1000.meta')
    saver.restore(sess, tf.train.latest_checkpoint("D:/sess/"))

    # get weights
    graph = tf.get_default_graph()
    fc2_w = graph.get_tensor_by_name("truediv:0")

    print(fc2_w.eval())


INFO:tensorflow:Restoring parameters from D:/sess/-1000
[[-0.06144057  0.02499304  0.00194109 ...  0.06388251 -0.00429929
   0.06115478]
 [ 0.05107442 -0.12265075  0.15208824 ...  0.07488727  0.00135686
  -0.02145557]
 [-0.10494224 -0.17098777  0.08450435 ... -0.07129572 -0.08744776
  -0.12084484]
 ...
 [-0.11192498 -0.12843628  0.14747913 ... -0.01921185 -0.09330991
  -0.13031763]
 [ 0.02682896 -0.12495162 -0.01443376 ... -0.07170907  0.11928383
   0.09584372]
 [-0.03799421 -0.03339894  0.00313361 ...  0.10551418  0.06171379
   0.05161681]]
